# A procedure to train and test target-specific machine-learning classifiers for structure-based virtual screening 

Please cite: Tran-Nguyen, V. K. & Ballester, P. J. A practical guide to machine-learning scoring for structure-based virtual screening. Nat. Protoc. (2022)

This is a user-friendly Jupyter notebook that guides you through different steps to train and evaluate a new target-specific machine-learning classifier for structure-based virtual screening. Please follow the instructions provided herein and in our Nature Protocols paper cited above while you use this notebook.

## 1. Import all necessary Python dependencies

Several Python dependencies need to be installed beforehand: set up your protocol-env environment using conda and the yml file protocol-env.yml (downloaded from our github repository) before runnning this Jupyter notebook.

In [ ]:
import os
import numpy as np
import pandas as pd
import oddt
import oddt.pandas as opd
from oddt.pandas import ChemDataFrame
from oddt.fingerprints import PLEC
from scipy import stats
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import matthews_corrcoef, precision_recall_curve, accuracy_score, auc
from sklearn.model_selection import cross_val_predict
from sklearn.neural_network import MLPClassifier
from sklearn.utils import parallel_backend
from xgboost.sklearn import XGBClassifier
from rdkit import Chem
from rdkit.Chem import AllChem
import deepchem as dc
from deepchem.utils import download_url, load_from_disk
from deepchem.utils.vina_utils import prepare_inputs
from deepchem.models import AtomicConvModel
from deepchem.feat import RdkitGridFeaturizer
from joblib import Parallel, delayed
from tqdm import tqdm
import glob
import tempfile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, regularizers
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Activation
from tensorflow.keras.optimizers import Adadelta, Adam, RMSprop
import hyperopt
from hyperopt import hp, tpe, Trials, fmin, STATUS_OK, space_eval

## 2. Load data tables for the training set and the test set 

Each data table (in the csv file format) needs to consist of these columns:

- "mol_name": provide the identifier (IDs) of each molecule
- "activity": state whether each molecule is an "Active" or an "Inactive"
- "potency": provide the potency value (if available) of each molecule: pIC50, pEC50, pKd, or pKi (all active concentrations in mol/l before logarithmic conversion)

More detailed instructions can be found in our Nature Protocols paper cited above.

Examples of these csv data files are provided along with this Jupyter notebook on github.

In [ ]:
#Provide the pathway to the csv training data file:
train_data = pd.read_csv("Provide_the_pathway_to_your_csv_training_data_file")

#Provide the pathway to the csv test data file:
test_data = pd.read_csv("Provide_the_pathway_to_your_csv_test_data_file")

#Call the "activity" labels of all training and test molecules:
Train_Class = train_data['activity']
Test_Class = test_data['activity']

## 3. Train and evaluate target-specific machine-learning classifiers 

Examples given in this notebook involve:

- A featurization scheme using protein-ligand extended connectivity (PLEC) fingerprints
- Five supervised machine-learning algorithms: random forest (RF), extreme gradient boosting (XGB), support vector machine (SVM), artificial neural network (ANN), and deep neural network (DNN)

### 3.1. Import training and test molecules along with target structure(s) 


- The structures of all docked molecules are in the mol2 (multi-mol2) file format.
- The target structure(s) is (are) in the mol2 file format. The target structure for training may be different from that for testing.



In [ ]:
#Provide the pathway to the training molecules (mol2 or multi-mol2):
train_mol2 = opd.read_mol2("Provide_the_pathway_to_the_(multi)mol2_file_of_your_training_molecules")
train_mol2.columns = ['mol', 'mol_name']
train = train_mol2.merge(train_data.drop_duplicates(subset = ['mol_name']), how = 'left', on = 'mol_name')

#Provide the pathway to the test molecules (mol2 or multi-mol2):
test_mol2 = opd.read_mol2("Provide_the_pathway_to_the_(multi)mol2_file_of_your_test_molecules")
test_mol2.columns = ['mol', 'mol_name']
test = test_mol2.merge(test_data.drop_duplicates(subset = ['mol_name']), how = 'left', on = 'mol_name')

#Extract the structures of all training and test molecules:
train_mols = train['mol']
test_mols = test['mol']

#Provide the pathway(s) to the training and set target/receptor structure(s):
receptor_train = next(oddt.toolkit.readfile('mol2', 'Provide_the_pathway_to_your_training_receptor_mol2_file'))
receptor_test = next(oddt.toolkit.readfile('mol2', 'Provide_the_pathway_to_your_test_receptor_mol2_file'))

### 3.2. Extract PLEC fingerprints from input data 

In [ ]:
#Users may keep only one of the following two functions if the training target/receptor structure is the same as the test target/receptor structure
#Users may change the parameters (size, depth_protein, depth_ligand, distance_cutoff) if they wish
def parallel_plec_train(mol):
    feature = PLEC(mol, protein = receptor_train, size = 4092, 
                   depth_protein = 4, depth_ligand = 2,
                   distance_cutoff = 4.5, sparse = False)
    return feature

def parallel_plec_test(mol):
    feature = PLEC(mol, protein = receptor_test, size = 4092, 
                   depth_protein = 4, depth_ligand = 2,
                   distance_cutoff = 4.5, sparse = False)
    return feature

#Users may choose another number of cores (num_cores) that corresponds to their computing resources
num_cores = 20
train_features = Parallel(n_jobs = num_cores, backend = "multiprocessing")(delayed(parallel_plec_train)(mol) for mol in tqdm(train_mols))
test_features = Parallel(n_jobs = num_cores, backend = "multiprocessing")(delayed(parallel_plec_test)(mol) for mol in tqdm(test_mols))

### 3.3. Train and test the RF algorithm 

In [ ]:
#Train the RF model on the training molecules:
rf_plec = RandomForestClassifier(n_estimators = 400, max_features = 'sqrt', n_jobs = 30, random_state = 0)
rf_plec.fit(train_features, Train_Class)

#Test the RF model on the test molecules:
prediction_test_rf_plec_class = rf_plec.predict(test_features)
prediction_test_rf_plec_prob = rf_plec.predict_proba(test_features)

In [ ]:
#Get virtual screening results on the test molecules and export the resulting hit list to a csv file:
plec_result_rf = pd.DataFrame({"Active_Prob": prediction_test_rf_plec_prob[:, 0],
                               "Inactive_Prob": prediction_test_rf_plec_prob[:, 1],
                               "Predicted_Class": prediction_test_rf_plec_class,
                               "Real_Class": Test_Class})
plec_result_rf.to_csv("Provide_the_pathway_to_a_directory_to_store_your_csv_hit_list")

#Compute precision, recall values and export the data table:
precision_plec_rf, recall_plec_rf, threshold_plec_rf = precision_recall_curve(Test_Class, plec_result_rf['Active_Prob'], pos_label = 'Active')
rf_plec_precision_recall = pd.DataFrame({"Precision": precision_plec_rf, "Recall": recall_plec_rf})
rf_plec_precision_recall.to_csv("Provide_the_pathway_to_a_directory_to_store_your_csv_precison_recall_data_table", index = False)

### 3.4. Train and test the XGB algorithm 

In [ ]:
#Train the XGB model on the training molecules:
xgb_plec = XGBClassifier(n_jobs = 40, random_state = 0)
xgb_plec.fit(np.array(train_features), Train_Class)

#Test the XGB model on the test molecules:
prediction_test_xgb_plec_class = xgb_plec.predict(np.array(test_features))
prediction_test_xgb_plec_prob = xgb_plec.predict_proba(np.array(test_features))

In [ ]:
#Get virtual screening results on the test molecules and export the resulting hit list to a csv file:
plec_result_xgb = pd.DataFrame({"Active_Prob": prediction_test_xgb_plec_prob[:, 0],
                                "Inactive_Prob": prediction_test_xgb_plec_prob[:, 1],
                                "Predicted_Class": prediction_test_xgb_plec_class,
                                "Real_Class": Test_Class})
plec_result_xgb.to_csv("Provide_the_pathway_to_a_directory_to_store_your_csv_hit_list")

#Compute precision, recall values and export the data table:
precision_plec_xgb, recall_plec_xgb, threshold_plec_xgb = precision_recall_curve(Test_Class, plec_result_xgb['Active_Prob'], pos_label = 'Active')
xgb_precision_recall_result = pd.DataFrame({"Precision": precision_plec_xgb, "Recall": recall_plec_xgb})
xgb_precision_recall_result.to_csv("Provide_the_pathway_to_a_directory_to_store_your_csv_precison_recall_data_table", index = False)

### 3.5. Train and test the SVM algorithm

In [ ]:
#Train the SVM model on the training molecules:
svm_plec = SVC(degree = 3, kernel = "rbf", probability = True, random_state = 0)
svm_plec.fit(train_features, Train_Class)

#Test the SVM model on the test molecules:
prediction_test_svm_plec_class = svm_plec.predict(test_features)
prediction_test_svm_plec_prob = svm_plec.predict_proba(test_features)

In [ ]:
#Get virtual screening results on the test molecules and export the resulting hit list to a csv file:
plec_result_svm  = pd.DataFrame({"Active_Prob": prediction_test_svm_plec_prob[:, 0],
                                 "Inactive_Prob": prediction_test_svm_plec_prob[:, 1],
                                 "Predicted_Class": prediction_test_svm_plec_class,
                                 "Real_Class": Test_Class})
plec_result_svm.to_csv("Provide_the_pathway_to_a_directory_to_store_your_csv_hit_list")

#Compute precision, recall values and export the data table:
precision_plec_svm, recall_plec_svm, threshold_plec_svm = precision_recall_curve(Test_Class, plec_result_svm['Active_Prob'], pos_label = 'Active')
svm_precision_recall_result = pd.DataFrame({"Precision": precision_plec_svm, "Recall": recall_plec_svm})
svm_precision_recall_result.to_csv("Provide_the_pathway_to_a_directory_to_store_your_csv_precison_recall_data_table", index = False)

### 3.6. Train and test the ANN algorithm 

In [ ]:
#Train the ANN model on the training molecules:
ann_plec = MLPClassifier(max_iter = 9000, random_state = 0)
ann_plec.fit(train_features, Train_Class)

#Test the ANN model on the test molecules:
prediction_test_ann_plec_class = ann_plec.predict(test_features)
prediction_test_ann_plec_prob = ann_plec.predict_proba(test_features)

In [ ]:
#Get virtual screening results on the test molecules and export the resulting hit list to a csv file:
plec_result_ann = pd.DataFrame({"Active_Prob": prediction_test_ann_plec_prob[:, 0],
                                "Inactive_Prob": prediction_test_ann_plec_prob[:, 1],
                                "Predicted_Class": prediction_test_ann_plec_class,
                                "Real_Class": Test_Class})
plec_result_ann.to_csv("Provide_the_pathway_to_a_directory_to_store_your_csv_hit_list")

#Compute precision, recall values and export the data table:
precision_plec_ann, recall_plec_ann, threshold_plec_ann = precision_recall_curve(Test_Class, plec_result_ann['Active_Prob'], pos_label = 'Active')
ann_precision_recall_result = pd.DataFrame({"Precision": precision_plec_ann, "Recall": recall_plec_ann})
ann_precision_recall_result.to_csv("Provide_the_pathway_to_a_directory_to_store_your_csv_precison_recall_data_table", index = False)

### 3.7. Train and test the DNN algorithm¶ 

In [ ]:
#Train the DNN model on the training molecules:
Activity_Dict = {"Active": 1, "Inactive": 0}
y_train = [Activity_Dict[item] for item in Train_Class]
y_test = [Activity_Dict[item] for item in Test_Class]
y_train = np.asarray(y_train).astype("float32")
y_test = np.asarray(y_test).astype("float32")
tf.random.set_seed(0)
dnn_plec = keras.Sequential([
    layers.Dense(8192, kernel_regularizer = regularizers.l2(0), activation = "relu"),
    layers.BatchNormalization(),
    layers.Dropout(0),
    layers.Dense(4069, activation = "relu", kernel_regularizer = regularizers.l2(0)),
    layers.BatchNormalization(),
    layers.Dense(2048, activation = "relu"),
    layers.Dropout(0),
    layers.Dense(1, activation = "sigmoid")
])
dnn_plec.compile(optimizer = "rmsprop", loss = "binary_crossentropy", metrics = ['accuracy'])
dnn_plec.fit(np.array(train_features), y_train, epochs = 100, batch_size = 500, verbose = False)

#Test the DNN model on the test molecules:
prediction_test_dnn_plec_prob = dnn_plec.predict(np.array(test_features))
prediction_test_dnn_plec_class = ["Active" if num >= 0.5 else "Inactive" for num in prediction_test_dnn_plec_prob]

In [ ]:
#Get virtual screening results on the test molecules and export the resulting hit list to a csv file:
plec_result_dnn = pd.DataFrame({"Active_Prob": prediction_test_dnn_plec_prob[:, 0],
                                "Predicted_Class": prediction_test_dnn_plec_class,
                                "Real_Class": Test_Class})
plec_result_dnn.to_csv("Provide_the_pathway_to_a_directory_to_store_your_csv_hit_list")

#Compute precision, recall values and export the data table:
precision_plec_dnn, recall_plec_dnn, threshold_plec_dnn = precision_recall_curve(Test_Class, plec_result_dnn['Active_Prob'], pos_label = 'Active')
dnn_precision_recall_result = pd.DataFrame({"Precision": precision_plec_dnn, "Recall": recall_plec_dnn})
dnn_precision_recall_result.to_csv("Provide_the_pathway_to_a_directory_to_store_your_csv_precison_recall_data_table", index = False)